import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [2]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, avg, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank
from pyspark.sql.functions import when
from pyspark.sql.functions import filter


PATH = 'ds_salaries.csv'

create local SparkSession

In [3]:
spark=SparkSession.builder.appName('Spark_Task').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

22/12/09 14:09:09 WARN Utils: Your hostname, U-inv1536 resolves to a loopback address: 127.0.1.1; using 10.202.32.26 instead (on interface wlp0s20f3)
22/12/09 14:09:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 14:09:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('work_year', StringType(), True),
    StructField('experience_level', StringType(), True),
    StructField('employment_type', StringType(), True),
    StructField('job_title', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('salary_currency', StringType(), True),
    StructField('salary_in_usd', IntegerType(), True),
    StructField('employee_residence', StringType(), True),
    StructField('remote_ratio', IntegerType(), True),
    StructField('company_location', StringType(), True),
    StructField('company_size', StringType(), True),
])

read csv with inferschema

In [5]:
df_schema = spark.read.csv(PATH, header=True, inferSchema=True)

write schema of scv on screen

In [6]:
df_schema.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
df = spark.read.csv(PATH, header=True, schema=schema)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df.show(5)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer| 85000|            GBP|       109024|                GB|          50|              GB|

print data in dataframe using display(df.toPandas())

In [10]:
df.toPandas()

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


In [11]:
display(df.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [12]:
df_job_title = df.select('job_title').distinct().toPandas().sort_values(by='job_title')

print all rows from df_job_titles without truncating jobs

In [13]:
df_job_title

,job_title
0,3D Computer Vision Researcher
13,AI Scientist
32,Analytics Engineer
45,Applied Data Scientist
36,Applied Machine Learning Scientist
47,BI Data Analyst
44,Big Data Architect
41,Big Data Engineer
34,Business Data Analyst
22,Cloud Data Engineer


create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [14]:
df_analytic = df.filter(col('salary_currency') == 'USD').groupBy('job_title', 'salary_currency').agg(
    avg('salary_in_usd').alias('avg_salary'), 
    min('salary_in_usd').alias('min_salary'), max('salary_in_usd').alias('max_salary'), 
    count('salary_in_usd').alias('count')
    )

print all rows from df_analytic without trancating jobs

In [15]:
df_analytic.toPandas()

,job_title,salary_currency,avg_salary,min_salary,max_salary,count
0,Data Analytics Manager,USD,127134.285714,105400,150260,7
1,Lead Data Scientist,USD,152500.000000,115000,190000,2
2,BI Data Analyst,USD,82454.400000,9272,150000,5
3,Research Scientist,USD,139428.428571,42000,450000,7
4,Data Engineer,USD,139465.800000,4000,324000,85
5,Computer Vision Software Engineer,USD,110000.000000,70000,150000,2
6,Lead Data Engineer,USD,154250.000000,56000,276000,4
7,Big Data Engineer,USD,49333.333333,18000,70000,3
8,AI Scientist,USD,79800.000000,12000,200000,5
9,Head of Data Science,USD,146718.750000,85000,224000,4


now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [16]:
windowSpec = Window.partitionBy().orderBy(col('avg_salary').desc())

df_analytic_sorted = df_analytic.withColumn('row_id', row_number().over(windowSpec))

print all data from df_analytic

In [17]:
df_analytic_sorted.toPandas()

,job_title,salary_currency,avg_salary,min_salary,max_salary,count,row_id
0,Data Analytics Lead,USD,405000.000000,405000,405000,1,1
1,Principal Data Engineer,USD,328333.333333,185000,600000,3,2
2,Financial Data Analyst,USD,275000.000000,100000,450000,2,3
3,ML Engineer,USD,263000.000000,256000,270000,2,4
4,Principal Data Scientist,USD,255500.000000,151000,416000,4,5
5,Director of Data Science,USD,247666.666667,168000,325000,3,6
6,Applied Data Scientist,USD,238000.000000,157000,380000,3,7
7,Head of Data,USD,221666.666667,200000,235000,3,8
8,Director of Data Engineering,USD,200000.000000,200000,200000,1,9
9,Machine Learning Infrastructure Engineer,USD,195000.000000,195000,195000,1,10


it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [20]:
#по идее можно пройтись итерацией по df_analytic_sorted.columns[0:5] но я не мог вспомнить как именно в пайспарке
df_analytic = df_analytic_sorted.select('row_id', 'job_title', 'salary_currency', 'avg_salary', 'min_salary', 'max_salary', 'count')

print df_analytic now

In [21]:
df_analytic.show()

+------+--------------------+---------------+------------------+----------+----------+-----+
|row_id|           job_title|salary_currency|        avg_salary|min_salary|max_salary|count|
+------+--------------------+---------------+------------------+----------+----------+-----+
|     1| Data Analytics Lead|            USD|          405000.0|    405000|    405000|    1|
|     2|Principal Data En...|            USD| 328333.3333333333|    185000|    600000|    3|
|     3|Financial Data An...|            USD|          275000.0|    100000|    450000|    2|
|     4|         ML Engineer|            USD|          263000.0|    256000|    270000|    2|
|     5|Principal Data Sc...|            USD|          255500.0|    151000|    416000|    4|
|     6|Director of Data ...|            USD|247666.66666666666|    168000|    325000|    3|
|     7|Applied Data Scie...|            USD|          238000.0|    157000|    380000|    3|
|     8|        Head of Data|            USD|221666.66666666666|    20

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [22]:
#можно ли тут сразу в фильр пропиххнуть оконку?
windowExp = Window.partitionBy('experience_level').orderBy(col('salary_in_usd').desc())
df_exp_level = df.withColumn('salary_rank_per_level', rank().over(windowExp)).filter(col('salary_rank_per_level') == 1)

print here df_exp_lvl

In [23]:
df_exp_level.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------------------+
| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|salary_rank_per_level|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+---------------------+
| 37|     2020|              EN|             FT|Machine Learning ...|250000|            USD|       250000|                US|          50|              US|           L|                    1|
|252|     2021|              EX|             FT|Principal Data En...|600000|            USD|       600000|                US|         100|              US|           L|                    1|
| 33|     2020|              MI|             

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [24]:
windowBest = Window.partitionBy('experience_level').orderBy(col('salary').desc())

df_best = df.withColumn('salary_rank_per_level', rank().over(windowBest)).filter(col('salary_rank_per_level') == 1).select("id", "experience_level", "salary", "employee_residence", "salary_rank_per_level")

print df_best

In [25]:
df_best.show(5)

+---+----------------+--------+------------------+---------------------+
| id|experience_level|  salary|employee_residence|salary_rank_per_level|
+---+----------------+--------+------------------+---------------------+
| 16|              EN| 4450000|                JP|                    1|
|384|              EX| 6000000|                IN|                    1|
|177|              MI|30400000|                CL|                    1|
|285|              SE| 7000000|                IN|                    1|
+---+----------------+--------+------------------+---------------------+



drop duplicates if exist by experience_level

In [26]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [27]:
df_best.show()

+---+----------------+--------+------------------+---------------------+
| id|experience_level|  salary|employee_residence|salary_rank_per_level|
+---+----------------+--------+------------------+---------------------+
| 16|              EN| 4450000|                JP|                    1|
|384|              EX| 6000000|                IN|                    1|
|177|              MI|30400000|                CL|                    1|
|285|              SE| 7000000|                IN|                    1|
+---+----------------+--------+------------------+---------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [28]:
df_new_best = df_best.drop('id').withColumn('desired_level', when(df.experience_level == 'MI', 'middle').when(df.experience_level == 'SE', 'senior').otherwise(None))

print df_new_best

In [29]:
df_new_best.show()

+----------------+--------+------------------+---------------------+-------------+
|experience_level|  salary|employee_residence|salary_rank_per_level|desired_level|
+----------------+--------+------------------+---------------------+-------------+
|              EN| 4450000|                JP|                    1|         null|
|              EX| 6000000|                IN|                    1|         null|
|              MI|30400000|                CL|                    1|       middle|
|              SE| 7000000|                IN|                    1|       senior|
+----------------+--------+------------------+---------------------+-------------+



write df_new_best like 1.csv and load then it to df_final

In [30]:
df_new_best.write.csv('df_new_best', header=True)
df_final = spark.read.csv('df_new_best', header=True, inferSchema=True)

AnalysisException: path file:/home/user/Desktop/Innowise_Spark_task/1_PySpark_Basics/df_new_best already exists.

print df_final

In [ ]:
df_final.show()

+----------------+--------+------------------+---------------------+-------------+
|experience_level|  salary|employee_residence|salary_rank_per_level|desired_level|
+----------------+--------+------------------+---------------------+-------------+
|              EN| 4450000|                JP|                    1|         null|
|              EX| 6000000|                IN|                    1|         null|
|              MI|30400000|                CL|                    1|       middle|
|              SE| 7000000|                IN|                    1|       senior|
+----------------+--------+------------------+---------------------+-------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [ ]:
df_final = df_final.na.drop().join(df, (df_final.salary == df.salary) & (df_final.employee_residence == df.employee_residence),'inner')

print df_final

In [ ]:
df_final.toPandas()

,experience_level,salary,employee_residence,salary_rank_per_level,desired_level,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,MI,30400000,CL,1,middle,177,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L
1,SE,7000000,IN,1,senior,285,2021,SE,FT,Data Science Manager,7000000,INR,94665,IN,50,IN,L


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [ ]:
A = df_final
A.select(col('salary_in_usd')).show()

+-------------+
|salary_in_usd|
+-------------+
|        40038|
|        94665|
+-------------+



It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system